# Transfer Learning 

##### Import libraries

In [13]:
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [14]:
from tensorflow.keras.datasets import mnist

In [15]:
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16

##### Enabling GPU (allow grow to let cuDNN function)

In [16]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

##### Extract a previously trained model

In [17]:
# Extract VGG16 from tensorflow.keras.applications library
prior_model = VGG16(weights='imagenet',include_top=False, input_shape=(64,64,3))
    # include_top=False : remove the last layer 
    # input_shape=(64,64,3) : specify input shape

### Building new model 1

##### Creating new model based on previous 

In [18]:
model = Sequential()
model.add(prior_model) # add VGG16 as a typical layer
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))

In [19]:
# model.summary() # VGG16 is considered as one single layer 

In [20]:
# model.layers[0].summary() # here to get details about VGG16

##### Freezing layers from previous model (total freeze)

In [21]:
for layers in model.layers[0].layers: # looping over each layers in layer 0 to freeze them
    layers.trainable = False

model.layers[0].trainable = False # freezing layer 0 as well for good measure

##### Compiling the model 

In [22]:
# compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Building new model 2 (higher resolution 128 x 128)

In [23]:
# loading previous model
PREVIOUS_MODEL_PATH = '../../_out_of_git/Deeplearning/64_by_64.h5'
previous_model = load_model(PREVIOUS_MODEL_PATH)

In [24]:
# previous_model.summary()

In [25]:
# previous_model.layers[0].summary()

In [26]:
# Building model HR (higher resolution)
modelHR = Sequential()
modelHR.add(Conv2D(64,kernel_size=(3,3),input_shape=(128,128,3),activation='relu', padding='same'))
modelHR.add(MaxPooling2D(pool_size=(2, 2)))

for layer in previous_model.layers[0].layers[2:]:
  # here we precise that we want to take all the layers from the second one to the last one in the VGG-16 model.
  modelHR.add(layer)

In [27]:
previous_model.layers[1:]

In [28]:
# Finally we add all the other layers in our previous model, while freezing them in the process
for layer in previous_model.layers[1:]:
  layer.trainable = False
  modelHR.add(layer)

In [29]:
modelHR.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)      

In [30]:
# Compiling model HR
modelHR.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

##### Creating image generator 

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# creating the image generator

generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_generator = ImageDataGenerator(
    rescale=1/255,
)

##### Creating train test sets

In [32]:
# defining the constants for the model training

BATCH_SIZE = 32
EPOCHS = 20
URL_TRAINING = '../../_out_of_git/Deeplearning/CatdogData/training_set' 
URL_TESTING = '../../_out_of_git/Deeplearning/CatdogData/test_set' 

# creating the train and test sets

train_set = generator.flow_from_directory(URL_TRAINING, target_size=(64,64), batch_size=BATCH_SIZE)
test_set = test_generator.flow_from_directory(URL_TESTING, target_size=(64,64), batch_size=BATCH_SIZE)

# creating the train and test sets with HR (128 x 128)

train_setHR = generator.flow_from_directory(URL_TRAINING, target_size=(128,128), batch_size=BATCH_SIZE)
test_setHR = test_generator.flow_from_directory(URL_TESTING, target_size=(128,128), batch_size=BATCH_SIZE)

 

Found 1589 images belonging to 2 classes.
Found 378 images belonging to 2 classes.
Found 1589 images belonging to 2 classes.
Found 378 images belonging to 2 classes.


##### Fitting the model

In [27]:
# fitting the model
with tf.device('/GPU:0'):
    model.fit_generator(train_set, steps_per_epoch=len(train_set.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_set, validation_steps=len(test_set.filenames)//BATCH_SIZE )

C:\Users\jcmeu\Anaconda3\envs\GPU-enabled\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
49/49 [==============================] - 22s 230ms/step - loss: 0.8656 - accuracy: 0.5733 - val_loss: 0.5658 - val_accuracy: 0.7102
Epoch 2/20
49/49 [==============================] - 3s 54ms/step - loss: 0.5498 - accuracy: 0.7146 - val_loss: 0.5007 - val_accuracy: 0.7358
Epoch 3/20
49/49 [==============================] - 3s 54ms/step - loss: 0.4945 - accuracy: 0.7601 - val_loss: 0.4878 - val_accuracy: 0.7614
Epoch 4/20
49/49 [==============================] - 3s 55ms/step - loss: 0.4851 - accuracy: 0.7607 - val_loss: 0.4927 - val_accuracy: 0.7756
Epoch 5/20
49/49 [==============================] - 3s 54ms/step - loss: 0.4929 - accuracy: 0.7391 - val_loss: 0.4993 - val_accuracy: 0.7528
Epoch 6/20
49/49 [==============================] - 3s 54ms/step - loss: 0.4689 - accuracy: 0.7845 - val_loss: 0.5280 - val_accuracy: 0.7614
Epoch 7/20
49/49 [==============================] - 3s 55ms/step - loss: 0.4893 - accuracy: 0.7622 - val_loss: 0.5306 - val_accuracy: 0.7415
Epoch 8/20


In [52]:
# fitting the model with HR (128 x 128)
with tf.device('/GPU:0'):
    modelHR.fit_generator(train_setHR, steps_per_epoch=len(train_setHR.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_setHR, validation_steps=len(test_setHR.filenames)//BATCH_SIZE )

Epoch 1/20
49/49 [==============================] - 11s 138ms/step - loss: 0.6967 - accuracy: 0.5074 - val_loss: 0.6806 - val_accuracy: 0.5511
Epoch 2/20
49/49 [==============================] - 6s 124ms/step - loss: 0.6957 - accuracy: 0.5163 - val_loss: 0.6634 - val_accuracy: 0.5938
Epoch 3/20
49/49 [==============================] - 6s 121ms/step - loss: 0.6735 - accuracy: 0.5769 - val_loss: 0.6642 - val_accuracy: 0.5852
Epoch 4/20
49/49 [==============================] - 6s 119ms/step - loss: 0.6861 - accuracy: 0.5515 - val_loss: 0.6510 - val_accuracy: 0.6250
Epoch 5/20
49/49 [==============================] - 6s 116ms/step - loss: 0.6733 - accuracy: 0.5826 - val_loss: 0.6546 - val_accuracy: 0.6278
Epoch 6/20
49/49 [==============================] - 6s 114ms/step - loss: 0.6877 - accuracy: 0.5634 - val_loss: 0.6332 - val_accuracy: 0.6506
Epoch 7/20
49/49 [==============================] - 6s 115ms/step - loss: 0.6659 - accuracy: 0.6001 - val_loss: 0.6268 - val_accuracy: 0.6335
Epoch

##### Testing the model 

In [29]:
import numpy as np
from tensorflow.keras.preprocessing import image

TEST_IMAGE_URL = './test_image_cat.jpg'

test_image = image.load_img( TEST_IMAGE_URL , target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)

[[6.177306e-20 1.000000e+00]]


In [30]:
import numpy as np
from tensorflow.keras.preprocessing import image

TEST_IMAGE_URL1 = './test_image_dog.jpg'

test_image1 = image.load_img( TEST_IMAGE_URL1 , target_size = (64, 64))
test_image1 = image.img_to_array(test_image1)
test_image1 = np.expand_dims(test_image1, axis = 0)
result1 = model.predict(test_image1)
print(result1)

[[0. 1.]]


##### Evaluate model

In [32]:
# model.evaluate(test_set, steps=BATCH_SIZE)
model.evaluate_generator(test_set, steps=BATCH_SIZE)

C:\Users\jcmeu\Anaconda3\envs\GPU-enabled\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.4828740954399109, 0.7486772537231445]

In [54]:
modelHR.evaluate_generator(test_setHR, steps=BATCH_SIZE)

[0.5755034685134888, 0.7010582089424133]

In [57]:
test_setHR

##### Save model

In [31]:
# time to save the model

PATH = '../../_out_of_git/Deeplearning/64_by_64.h5'

model.save(PATH)